# Codestral fine-tuning with color dataset

2024-12-02 21:20

Attempt to replicate the color fine-tuning on the Codestral model. Unfortunately predict_with_generate is still work-in-progress so I used a merged version of the pull requests in both the `transformers` and `trl` repositories.

PRs merged:
https://github.com/huggingface/transformers/pull/32346
https://github.com/huggingface/trl/pull/2310
https://github.com/huggingface/trl/pull/2311

The output is really strange, makes no sense. The output from the original model is just the model rejecting to do anything at all.
I'd like to try to run it with the original versions of the merged repos.

In [1]:
!pip install -U git+https://github.com/tcz/transformers
!pip install -U git+https://github.com/huggingface/accelerate.git@8ade23cc6aec7c3bd3d80fef6378cafaade75bbe
!pip install -U git+https://github.com/huggingface/datasets.git@01f91bae037c98f2e05456287bab21470adb8f07
!pip install -U git+https://github.com/huggingface/evaluate.git@55f1bc6e072b05c2d9db1589a07e20f38902b1ec
!pip install -U git+https://github.com/huggingface/safetensors.git@f5839b6aee407652aa3078d91206b618dd84e3c2
!pip install -U git+https://github.com/huggingface/peft.git@eaaf03c12767c9d3dbc9edca7766cfcb799c38a0
!pip install -U git+https://github.com/tcz/trl.git

!pip install -U "bitsandbytes==0.43.2"

!pip install -U sentencepiece
!pip install pytest-playwright
!playwright install
!pip install matplotlib
!pip install pillow
!pip install torchvision
!pip install lpips

!playwright install-deps  
!pip install tensorboard

  Cloning https://github.com/tcz/transformers to /tmp/pip-req-build-rrvwhku3
  Running command git clone --filter=blob:none --quiet https://github.com/tcz/transformers /tmp/pip-req-build-rrvwhku3
  Resolved https://github.com/tcz/transformers to commit 69a8568b9b348e4b37c73bad9f9a08ea73fc80ae
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.6/447.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 141.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10102224 sha256=27ff7b28b2fb43f9ad84dfce2ea447f8f7cd999ad041c0d4bf80019054cd4b55

In [2]:
import torch
assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'

# on a multi-gpu machine
!FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE pip install flash-attn --no-build-isolation

# NOTE: use when 'Hardware not supported for Flash Attention'
# on a single gpu or only cpu machine 
! pip install ninja packaging
! MAX_JOBS=4 pip install flash-attn --no-build

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 75.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 22.8 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.7.0.post2-py3-none-any.whl size=183297046 sha256=b81d7374a7427805af50957ee26ed2575f4b7f21d85d8ae3f409e5387d01f340
  Stored in directory: /root/.cache/pip/wheels/dd/35/a4/958e31aa700d7f22a16771a3964aec266cd1d01ce62d6e16b8
Successfully built flash-attn


In [3]:
!git config --global credential.helper store

In [4]:
from huggingface_hub import login
 
login(
  token="<HF_API_KEY_REMOVED>", 
  add_to_git_credential=True
)

In [5]:
!apt install zip -y
!rm -rf data-rb-color
!mkdir -p data-rb-color
!wget "https://www.dropbox.com/scl/fi/vd0ypt9mo9oh0p9tf90h3/dataset-rb-color-fixed.zip?rlkey=bieseudpp5pzko5j4u1n67phq&dl=1" -O model.zip
!unzip model.zip -d data-rb-color

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  unzip
The following NEW packages will be installed:
  unzip zip
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 350 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 unzip amd64 6.0-26ubuntu3.2 [175 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 zip amd64 3.0-12build2 [176 kB]
Fetched 350 kB in 1s (661 kB/s)m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 34390 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-26ubuntu3.2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 11%] [######................................

In [6]:
from datasets import load_from_disk
dataset = load_from_disk('data-rb-color')
dataset = dataset.train_test_split(test_size=4/len(dataset))

dataset

DatasetDict({
    train: Dataset({
        features: ['svg', 'html'],
        num_rows: 99996
    })
    test: Dataset({
        features: ['svg', 'html'],
        num_rows: 4
    })
})

In [7]:
mistral_instruct_template = "[INST]{instruction}[/INST]"

system_prompt = """Your job is to turn an input SVG file to HTML and CSS code.
You must generate only HTML and CSS code, no additional text."""

def format_dataset(sample):
    instruction = f"{system_prompt}\n\n" + sample["svg"] + "\n\n"
    sample["prompt"]  = mistral_instruct_template.format(instruction=instruction)
    sample["completion"] = sample["html"]
    return sample

# convert dataset to instruct prompt template
columns_to_remove = list(dataset["train"].column_names)
dataset = dataset.map(format_dataset, remove_columns=columns_to_remove, batched=False)

Map:   0%|          | 0/99996 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [8]:
print(dataset['train'][0])

{'prompt': '[INST]Your job is to turn an input SVG file to HTML and CSS code.\nYou must generate only HTML and CSS code, no additional text.\n\n<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="style1"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="377" height="836" x="8" y="8" fill="rgb(178, 52, 154)"/></g><g data-tag="style" id="style1" data-z-index="auto" data-stacking-context="true"/></g></g></svg>\n\n[/INST]', 'completion': '<body></body>\n\n<style>\n\n        body {\n            background-color: #b2349a;\n        }\n    \n</style>'}


In [9]:
!mkdir -p sft_cache
!mkdir -p sft_cache/checkpoints
!mkdir -p sft_cache/model
!mkdir -p sft_cache/offload

In [10]:
### model
model_id = "mistral-community/Codestral-22B-v0.1"

### qlora related
r = 64
lora_alpha = 16
lora_dropout = 0.1
task_type = "CAUSAL_LM"

### bitsandbytes related
load_in_4bit=True
bnb_4bit_use_double_quant=True
bnb_4bit_quant_type="nf4"
bnb_4bit_compute_dtype="bfloat16"

### training related
output_dir = "sft_cache/checkpoints"
save_model_dir = "sft_cache/model/"
offload_folder = "sft_cache/offload"
log_dir=f"{output_dir}/logs"

num_train_epochs = 1

per_device_train_batch_size = 1
per_device_eval_batch_size = 1
gradient_accumulation_steps = 1
gradient_checkpointing = True

bf16 = True
fp16 = False

max_grad_norm = 0.3
weight_decay = 0.001
optim = "adamw_torch"

learning_rate = 2e-4
warmup_ratio = 0.03
lr_scheduler_type = "constant"

save_strategy = "no"
logging_steps = 25
logging_strategy = "steps"
group_by_length = True

max_seq_length = 4096
packing = False

In [11]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

tokenizer_config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [13]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
    bnb_4bit_compute_dtype=getattr(torch, bnb_4bit_compute_dtype),
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_cache=False if gradient_checkpointing else True,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False if gradient_checkpointing else True
model.config.pretraining_tp = 1 # num_of_gpus
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/41.8k [00:00<?, ?B/s]

model-00001-of-00009.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00009.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00009.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

model-00007-of-00009.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00008-of-00009.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00009-of-00009.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [14]:
import bitsandbytes as bnb
from peft import LoraConfig

In [15]:
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)


# get lora target modules
target_modules = find_all_linear_names(model)

In [16]:
lora_config = LoraConfig(
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=target_modules,
    bias="none",
    task_type=task_type,
)

In [17]:
from transformers import TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [18]:
# checkout for more info: Train on completions only https://huggingface.co/docs/trl/en/sft_trainer

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['prompt'])):
        text = f"{example['prompt'][i]}\n\n ### Answer: {example['completion'][i]}"
        output_texts.append(text)
    return output_texts

collator = DataCollatorForCompletionOnlyLM(
    response_template="### Answer:", 
    tokenizer=tokenizer
)

In [19]:
import numpy as np
from utils.similarity import calculate_metrics
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import torch

def add_image_to_tensorboard(name, step, img_path):
    image = Image.open(img_path)
    image = image.convert('RGB')
    image_array = np.array(image)
    image_tensor = torch.from_numpy(image_array)
    image_tensor = image_tensor.permute(2, 0, 1)
    image_tensor = image_tensor.float() / 255.0
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_image(name, image_tensor, step)
    
def add_text_to_tensorboard(name, step, text):
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_text(name, text, step)

def create_compute_metrics(trainer):
    def compute_metrics(prediction_dict):
        try:
            predictions = prediction_dict.predictions
            predictions[predictions == -100] = tokenizer.pad_token_id
            labels = prediction_dict.label_ids
            labels[labels == -100] = tokenizer.pad_token_id
        
            # Decode and do magic for metrics
            predictions = tokenizer.batch_decode(predictions)
            labels = tokenizer.batch_decode(labels)

            predictions = predictions[-4:]
            labels = labels[-4:]
            
            print("Predictions: ", predictions)
            print("Labels: ", labels)

            similarity_scores = []
            perceptual_losses = []
            index = 1
            for prediction, label in zip(predictions, labels):
                add_text_to_tensorboard(f'valid_{index}_label_text', trainer.state.global_step, label[0])
                add_text_to_tensorboard(f'valid_{index}_prediction_text', trainer.state.global_step, prediction)
                
                metrics = calculate_metrics(prediction, label[0])
                
                if metrics is not None:
                    similarity_scores.append(metrics['similarity'])
                    perceptual_losses.append(metrics['perceptual_loss'])
                    
                    add_image_to_tensorboard(f'valid_{index}_expectation', trainer.state.global_step, metrics['expected_screenshot_path'])
                    add_image_to_tensorboard(f'valid_{index}_prediction', trainer.state.global_step, metrics['predicted_screenshot_path'])
                
                index += 1

            result = {
                'similarity': float(np.mean(similarity_scores)),
                'perceptual_loss': float(np.mean(perceptual_losses)),
            }

            result = {k: round(v, 4) for k, v in result.items()}
            return result
        except Exception as e:
            print("Error during evaluation: ", str(e))
            print("Eval predictions: ", prediction_dict)
            
            return {
                'similarity': np.nan,
                'perceptual_loss': np.nan,
            }

    return compute_metrics

In [20]:
# set training arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    eval_strategy="steps",
    eval_steps=1000,
    # max_steps=max_steps,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    weight_decay=weight_decay,    
    optim=optim,
    learning_rate=learning_rate,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    save_strategy=save_strategy,
    logging_steps=logging_steps,
    logging_strategy=logging_strategy,
    group_by_length=group_by_length,
    # predict_with_generate=True,
)

In [21]:
# initialize sft trainer
trainer = SFTTrainer(
    args=training_arguments,
    model=model,
    peft_config=lora_config,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset["test"],
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    #max_seq_length=max_seq_length,
    #packing=packing,
    compute_metrics=None,
)

# trainer.compute_metrics = create_compute_metrics(trainer)


/opt/conda/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:255: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/99996 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [22]:
trainer.train()

/opt/conda/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
1000,0.067800,0.007870
2000,0.076400,0.099995
3000,0.022500,0.028387
4000,0.019800,0.070743
5000,0.064600,0.076111
6000,0.193000,1.811580
7000,0.094400,0.261919
8000,0.184900,0.059060
9000,0.150600,0.458487
10000,0.331300,0.296751


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=99996, training_loss=0.5561626512183085, metrics={'train_runtime': 89613.0911, 'train_samples_per_second': 1.116, 'train_steps_per_second': 1.116, 'total_flos': 4.832916589529825e+18, 'train_loss': 0.5561626512183085, 'epoch': 1.0})

In [23]:
trainer.model.save_pretrained(output_dir, safe_serialization=False)

In [26]:
text = dataset['test'][0]
text

{'prompt': '[INST]Your job is to turn an input SVG file to HTML and CSS code.\nYou must generate only HTML and CSS code, no additional text.\n\n<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="style1"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="377" height="836" x="8" y="8" fill="rgb(138, 240, 21)"/></g><g data-tag="style" id="style1" data-z-index="auto" data-stacking-context="true"/></g></g></svg>\n\n[/INST]',
 'completion': '<body></body>\n\n<style>\n\n        body {\n            background-color: #8af015;\n        }\n    \n</style>'}

In [29]:
# clear memory
del model
del trainer
torch.cuda.empty_cache()

#### step 6: merge adapter weights and base model

In [30]:
from peft import AutoPeftModelForCausalLM

In [31]:
# load PEFT model in fp16
model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,  # ATTENTION: This allows remote code execution
)  

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [32]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 6144)
        (layers): ModuleList(
          (0-55): 56 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=6144, out_features=6144, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=6144, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): l

In [33]:
# merge
merged_model = model.merge_and_unload()

In [34]:
print(merged_model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 6144)
    (layers): ModuleList(
      (0-55): 56 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=6144, out_features=6144, bias=False)
          (k_proj): Linear(in_features=6144, out_features=1024, bias=False)
          (v_proj): Linear(in_features=6144, out_features=1024, bias=False)
          (o_proj): Linear(in_features=6144, out_features=6144, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=6144, out_features=16384, bias=False)
          (up_proj): Linear(in_features=6144, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=6144, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((6144,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((6144,), eps=1e-05)
     

In [35]:
# save merged model
merged_model.save_pretrained(save_model_dir, safe_serialization=True,  max_shard_size="2GB")

In [36]:
# save tokenizer for easy inference
tokenizer.save_pretrained(save_model_dir)

('sft_cache/model/tokenizer_config.json',
 'sft_cache/model/special_tokens_map.json',
 'sft_cache/model/tokenizer.model',
 'sft_cache/model/added_tokens.json',
 'sft_cache/model/tokenizer.json')

In [37]:
del model
del merged_model
del tokenizer

torch.cuda.empty_cache()

### 5. Test and evaluate

In [38]:
# NOTE: restart the kernel and run from this section

#### prepare test dataset

In [39]:
# uncomment the test dataset and run all the cells within section 3: Create and prepare dataset

#### inference: finetuned model

In [40]:
import gc, torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.cuda.empty_cache()
gc.collect()

94

In [41]:
model_local_path = "sft_cache/model/"
print(f"model_local_path: {model_local_path}")

model_local_path: sft_cache/model/


In [42]:
tokenizer = AutoTokenizer.from_pretrained(
    model_local_path, trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

sft_model = AutoModelForCausalLM.from_pretrained(
    model_local_path,
    trust_remote_code=True,
    attn_implementation="flash_attention_2",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/23 [00:00<?, ?it/s]

In [43]:
eval_sample = dataset['test'][0]
eval_prompt, eval_completion = eval_sample["prompt"], eval_sample["completion"]

print(f"prompt: {eval_prompt}")
print("\n", f"*"*25, "\n")
print(f"completion: {eval_completion}")

prompt: [INST]Your job is to turn an input SVG file to HTML and CSS code.
You must generate only HTML and CSS code, no additional text.

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="style1"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="377" height="836" x="8" y="8" fill="rgb(138, 240, 21)"/></g><g data-tag="style" id="style1" data-z-index="auto" data-stacking-context="true"/></g></g></svg>

[/INST]

 ************************* 

completion: <body></body>

<style>

        body {
            background-color: #8af015;
        }
    
</style>


In [44]:
model_inputs = tokenizer([eval_prompt], return_tensors="pt").to("cuda")
sft_model.eval()
with torch.no_grad():
    generated_ids = sft_model.generate(
        **model_inputs, max_new_tokens=1000, do_sample=True,
        length_penalty=-5.0, repetition_penalty=2.0, num_beams=10,
    )
    results = tokenizer.batch_decode(generated_ids)[0]
    # prompt_length = model_inputs['input_ids'].shape[1]
    # results = tokenizer.batch_decode(generated_ids[prompt_length:])[0]
    print(results)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s>[INST] Your job is to turn an input SVG file to HTML and CSS code.
You must generate only HTML and CSS code, no additional text.

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="style1"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="377" height="836" x="8" y="8" fill="rgb(138, 240, 21)"/></g><g data-tag="style" id="style1" data-z-index="auto" data-stacking-context="true"/></g></g></svg>

[/INST]<></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></></>

#### inference: original model

In [45]:
del sft_model
del tokenizer

In [46]:
import gc, torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.cuda.empty_cache()
gc.collect()

20

In [47]:
model_id = "mistral-community/Codestral-22B-v0.1"
print(f"model_id: {model_id}")

model_id: mistral-community/Codestral-22B-v0.1


In [48]:
tokenizer = AutoTokenizer.from_pretrained(
    model_local_path, trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    attn_implementation="flash_attention_2",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [49]:
eval_sample = dataset['test'][0]
eval_prompt, eval_completion = eval_sample["prompt"], eval_sample["completion"]

print(f"prompt: {eval_prompt}")
print("\n", f"*"*25, "\n")
print(f"completion: {eval_completion}")

prompt: [INST]Your job is to turn an input SVG file to HTML and CSS code.
You must generate only HTML and CSS code, no additional text.

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="style1"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="377" height="836" x="8" y="8" fill="rgb(138, 240, 21)"/></g><g data-tag="style" id="style1" data-z-index="auto" data-stacking-context="true"/></g></g></svg>

[/INST]

 ************************* 

completion: <body></body>

<style>

        body {
            background-color: #8af015;
        }
    
</style>


In [50]:
model_inputs = tokenizer([eval_prompt], return_tensors="pt").to("cuda")
base_model.eval()
with torch.no_grad():
    generated_ids = base_model.generate(
        **model_inputs, max_new_tokens=32_000, do_sample=True
    )
    results = tokenizer.batch_decode(generated_ids)[0]
    print(results)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s>[INST] Your job is to turn an input SVG file to HTML and CSS code.
You must generate only HTML and CSS code, no additional text.

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="style1"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="377" height="836" x="8" y="8" fill="rgb(138, 240, 21)"/></g><g data-tag="style" id="style1" data-z-index="auto" data-stacking-context="true"/></g></g></svg>

[/INST] I understand your requirement, but it should be noted that I am not able to directly convert SVG to HTML/CSS as I am a language model that executes Python code and can interact using text-based

In [52]:
!ls -lh $output_dir

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


total 1.5G
-rw-r--r-- 1 root root 5.0K Dec  2 20:04 README.md
-rw-r--r-- 1 root root  787 Dec  2 20:04 adapter_config.json
-rw-r--r-- 1 root root 1.5G Dec  2 20:04 adapter_model.bin
drwxr-xr-x 3 root root   49 Dec  1 19:10 runs
